# Shockwave Traffic Jam

## Brief
L'addestramento consiste nel miglioramento dei parametri del modello a seguito di un batch di simulazioni.

### Make the dataset
Ogni dato del dataset è ottenuto stabilendo casualmente:
1. Lunghezza della strada
2. Numero di veicoli
3. Parametri per **OVM**
4. Parametri per **FTL**
5. Bilanciamento tra **OVM** e **FTL**
Si crea così un dato senza addensamenti
Le velocità sono ottenute in accordo con il modello **OVM**.

### Data Augmentation
Si aumentano il numero di dati in modo casuale proponendo una perturbazione delle velocità e un tempo di simulazione iniziale prima di attivare l'apprendimento.

Ad ogni chiamata del training quindi ci sarà l'aggiunta di rumore.

### Training
Si addestra la rete con due sistemi di premi:
1. feedback ad ogni tempo $\tau$ che suggeriscono alla rete quanto il suo attuale andamento è gradevole (accelerazioni contenute):
   1. Si tratta di un fine-tuning degli ultimi layer (più istintivi).
   2. Il simulatore prenderà traccia dell'accelerazione più forte (in modulo).
2. premio finale al tempo $T$ che indica alla rete lo spazio percorso complessivo:
   1. Si aggiorna il ragionamento e la memoria della rete, si tratta di un tuning dei layer più superficiali.
   2. Il simulatore prenderà traccia dello spazio percorso totale dal veicolo autonomo.

## The model
Il modello non ha accesso completo a tutta la strada, tuttavia può ricordare quello che vede e quello che ha fatto.

In questo modo in una strada lineare semplicemente il modello assocerà ciò che vede ad un'esperienza antecedente e anche se non è vero che quanto farà lo aiuterà, agirà comunque come se fosse per il bene proprio.

Per questa ragione usiamo delle RNN per l'addestramento di alto livello, semplicemente la rete memorizzerà la qualità della strategia nelle RNN

L'interpretazione delle strategie sarà fatta a livelli più profondi che ne stimano l'applicazione e che saranno addestrati con fine-tuning

# Make the dataset

Il dataset generato è solo una radice di costruzione del vero dataset usato per l'addestramento.

Generiamo quindi questa radice che prende in input:
- range di valori per la densità
- range di valori per $d_0$ richiesto da OVM
- range di valori per $\Delta$

La radice propone una distribuzione uniforme dei veicoli in uno stato stazionario.

La stabilità dello stato stazionario in un modello **OVM** **FTL** si ha per velocità ideale che rispetta la seguente relazione

$$
V'(L/N) < \frac{1}{2}\frac{\beta/\tau}{1+\beta} + \frac{1/\text{FTL}(d_0)^{1+\gamma}}{1+\beta}\frac{1}{(L/N)^{1+\gamma}}
$$

Il dataset viene creato affinché solitamente questa proprietà non si rispetti e che quindi ci sia instabilità, tuttavia non è da escludere il contrario.

In [ ]:
# MAKE DATASET ROOT
from source import ShockwaveTrafficJam
import torch
import os

dataset_path = 'data/db/TRAFFIC'
os.makedirs(dataset_path, exist_ok=True)

ShockwaveTrafficJam.dataset.make_traffic(1_024, torch.device('cuda'), 'data/db/TRAFFIC')

Il dataset è quindi un tensore avente 3 colonne indicanti:
- densità
- d0_OVM
- Delta

In [ ]:
# LOAD DATASET ROOT
from source import ShockwaveTrafficJam

dataset = ShockwaveTrafficJam.dataset.Dataset(dataset_path, root=True).to(torch.device('cuda'))

Il dataloader carica la radice e viene usato per produrre facilmente i batch.

Ogni batch presenta un'informazione completa e parallelizzata:
- posizioni dei veicoli : tensore di shape (batch_size, n_vehicles)
- velocità dei veicoli : tensore di shape (batch_size, n_vehicles)
- len_road, d0_OVM, Delta, Vmax, tau, d0_FTL, gamma, beta : tensore di shape (batch_size, 8)

In particolare sarà necessario passare al dataloader:
- range per il numero di veicoli : usato per generare il numero di veicoli del batch
- range per Vmax, tau, d0_FTL, gamma, beta
  
Sarà inoltre aggiunto un trasformatore dei dati che fungerà da data augmentation.

In [ ]:
# CREATE DATALOADER ROOT
from source import ShockwaveTrafficJam

transformer = ShockwaveTrafficJam.transforms.Compose([
    ShockwaveTrafficJam.transforms.RandomDropout(0.1),                # 10% of the data will be dropped
    ShockwaveTrafficJam.transforms.RandomNoise(1.0, 1.2),             # perturbation of position and speed
    #ShockwaveTrafficJam.transforms.Simulation(1.0/24.0, 24*60),       # simulate the traffic jam (60 sec) with 24FPS
])

dataloader = ShockwaveTrafficJam.dataloader.DataLoader(dataset, batch_size=8, shuffle=True, transform=transformer, root=True).to(torch.device('cuda'))

Visualizzo l'evoluzione dell'energia cinetica

In [ ]:
# SHOW A VIDEO
from source import ShockwaveTrafficJam

sim = next(iter(dataloader))

ShockwaveTrafficJam.simulator.auto_show_video(sim, 1/24, 24*60)

In [ ]:
# STUDY THE ENERGY
import pandas as pd

# creo un data frame dove salvo i dati
df = pd.DataFrame(columns=['time', 'energy', 'velocity'])

sim = next(iter(dataloader))

for i in range(24*60*100):
    sim.auto_step(1/24)
    energy = sim.energy()
    velocity = (sim.v.mean(dim=1)/sim.Vmax.reshape(-1)).mean().item()
    if velocity < 0:
        pass
    df.loc[i] = [i/30, (energy).mean().item(), velocity]

df.to_csv('data/out/energy.csv', index=False)

In [ ]:
sim.visual()

Uso il dataloader per creare il dataset completo che sarà poi usato per l'addestramento

In [ ]:
# MAKE DATASET
from tqdm.notebook import tqdm

for i, sim in tqdm(enumerate(dataloader), total=len(dataloader), leave=False):
    sim = sim.to(torch.device('cpu'))
    sim.save('data/db/TRAFFIC')

# Open dataset

E' possibile aprire direttamente il nuovo dataset con tutti gli scenari costruiti.

Ogni file rappresenta una parte del dataset, il dataloader carica i file in RAM e poi li smaltisce per l'addestramento.



In [ ]:
# LOAD the dataset
import torch
from source import ShockwaveTrafficJam

dataset = ShockwaveTrafficJam.dataset.Dataset('data/db/TRAFFIC', root=False)
dataloader = ShockwaveTrafficJam.dataloader.DataLoader(dataset, batch_size=16)

In [ ]:
# visualize a single data
dataloader = iter(dataloader)
sim = next(dataloader)
sim.visual()

## About the model

La rete fa uso di una `CNN` applicata alle misurazioni effettuate a $30$FPS, viene fornita una serie temporale di $316$ step (circa $10$ secondi) con $2$ canali:
- distanza relativa
- velocità relativa

La sequenza viene poi semplificata ottenendo $64$ sequenze a $16$ step (con un frame rate simile al tempo di reazione umano : $0.5$ secondi).

Dopodiché viene passato il risultato attraverso un `trasformatore` che ne deduce il contesto che fa uso di una maschera **look-ahead** per mantenere la progressività dei dati.

Infine la serie viene poi linearizzata e classificata con un `MLP`.

Il modello deve prevedere la densità media globale dell'intero esperimento. Infatti usando la sua previsione, basata sull'esperienza di guida, si può applicare il modello risolutivo.

La soluzione infatti prevede che la velocità ideale dei veicoli sia valutata sulla densità media globale.


In [ ]:
from source.AC import roboCar

agent = roboCar().to(torch.device('cuda'))

print(f"Trainable parameters: {sum(p.numel() for p in agent.parameters() if p.requires_grad)}")

## Pre training

Come detto il modello deve prevedere la densità media globale, per questa ragione il suo output sarà un numero reale positivo (attivato quindi con ReLU) che indica la distanza media tra i veicoli.

La rete quindi prosegue con determinati pesi per un certo lasso di tempo, al termine del quale verrà calcolata l'energia cinetica globale.

L'energia cinetica tende ad aumentare poiché ogni veicolo proverà 

In [ ]:
from source import training

# estrapolo un batch dal dataloader
training.train(
    agent,
    dataloader,
    optimizer=torch.optim.Adam(agent.parameters(), lr=1e-4),
    epochs=2,
    time_step = 1/30,
    steps = 3,
    deep_steps=600,
    with_reset=True)

Salvo il pretraining così da poterlo riaprire senza perderlo

In [ ]:
agent.save_weights('data/models/pre_trained.pth')